## Task 3

In [1]:
## importing the required libraries
import pandas as pd

In [2]:
## loading the dataset
df = pd.read_excel('rawdata.xlsx')

In [3]:
## checking the first five row of data
df.head()

,date,time,sensor,location,number,activity,position,location.1
0,2024-01-16,10:10:30,0.0,A1,1,placed,inside,A1
1,2024-01-16,10:12:30,1.0,A2,1,picked,inside,A2
2,2024-01-16,10:28:30,1.0,A3,1,picked,Inside,A3
3,2024-01-16,10:38:20,0.0,A4,1,placed,Inside,A4
4,2024-01-16,10:45:20,1.0,A5,1,placed,inside,A5


In [4]:
print(f'Dataset shape: {df.shape}')

Dataset shape: (175, 8)


In [5]:
print(f'Dataset data Types : \n{df.dtypes}')

Dataset data Types : 
date          datetime64[ns]
time                  object
sensor               float64
location              object
number                 int64
activity              object
position              object
location.1            object
dtype: object


In [6]:
## checking the null value in the dataset
df.isnull().sum()

date          0
time          0
sensor        1
location      0
number        0
activity      0
position      0
location.1    0
dtype: int64

In [7]:
## seeing the null vale column
df[df['sensor'].isnull()]

,date,time,sensor,location,number,activity,position,location.1
150,2024-01-18,19:10:00,NaN,A31,1,placed,Inside,A31


In [8]:
## Dropping the null value column
df.dropna(axis=0, inplace=True)

In [9]:
## sorting the dataset 
df = df.sort_values(by=['date','time'])

In [10]:
df.head()

,date,time,sensor,location,number,activity,position,location.1
16,2024-01-16,00:02:20,1.0,A17,1,placed,inside,A17
24,2024-01-16,00:02:20,0.0,A25,1,placed,inside,A25
18,2024-01-16,00:08:20,1.0,A19,1,picked,inside,A19
19,2024-01-16,00:10:20,0.0,A20,1,placed,inside,A20
20,2024-01-16,00:15:20,0.0,A21,1,placed,inside,A21


In [11]:
## creating the new column name: datetime  by merging the date and time column
df['datetime'] = pd.to_datetime(df['date'].astype(str) + ' ' + df['time'].astype(str))

In [12]:
## cheking the unique value in positin column
df['position'].unique()

array(['inside', 'Inside', 'outside'], dtype=object)

In [13]:
## making the postion column data in lower case
df['position'] = df['position'].str.lower()


In [14]:
## creatinf the new column duration: groupby date and position and calculating the difference ad=nd then converting into minutes
df['duration'] = df.groupby(['date', 'position'])['datetime'].diff().dt.total_seconds().div(60)


In [15]:
df.head()

,date,time,sensor,location,number,activity,position,location.1,datetime,duration
16,2024-01-16,00:02:20,1.0,A17,1,placed,inside,A17,2024-01-16 00:02:20,NaN
24,2024-01-16,00:02:20,0.0,A25,1,placed,inside,A25,2024-01-16 00:02:20,0.0
18,2024-01-16,00:08:20,1.0,A19,1,picked,inside,A19,2024-01-16 00:08:20,6.0
19,2024-01-16,00:10:20,0.0,A20,1,placed,inside,A20,2024-01-16 00:10:20,2.0
20,2024-01-16,00:15:20,0.0,A21,1,placed,inside,A21,2024-01-16 00:15:20,5.0


In [16]:
## separating the inisde data and outise data
inside_data = df[df['position'] == 'inside']
outside_data = df[df['position'] == 'outside']

In [17]:
## calculating the total inside duration datewise
inside_duration = inside_data.groupby('date')['duration'].sum().reset_index()
inside_duration.columns = ['date', 'total_inside_duration']

In [18]:
inside_duration.head()

,date,total_inside_duration
0,2024-01-16,717.000000
1,2024-01-17,190.833333
2,2024-01-18,559.833333


In [19]:
## calculating the total outside duration datewise
outside_duration = outside_data.groupby('date')['duration'].sum().reset_index()
outside_duration.columns = ['date', 'total_outside_duration']


In [20]:
outside_duration.head()

,date,total_outside_duration
0,2024-01-18,357.833333


In [21]:
## merging the inside and outside duration
total_duration = pd.merge(inside_duration, outside_duration, on='date', how='outer')


In [22]:
total_duration

,date,total_inside_duration,total_outside_duration
0,2024-01-16,717.000000,NaN
1,2024-01-17,190.833333,NaN
2,2024-01-18,559.833333,357.833333


In [23]:
## for checking the unique value in the activity column
df['activity'].unique()

array(['placed', 'picked'], dtype=object)

In [24]:
## calculating the total picking and placing count
picking_count = df[df['activity'] == 'picked'].groupby('date').size().reset_index(name='picking_count')
placing_count = df[df['activity'] == 'placed'].groupby('date').size().reset_index(name='placing_count')


In [25]:
picking_count, placing_count

(        date  picking_count
 0 2024-01-16             40
 1 2024-01-17             10
 2 2024-01-18             37,
         date  placing_count
 0 2024-01-16             40
 1 2024-01-17              9
 2 2024-01-18             38)

In [26]:
## merging the picking and placing count
activity_counts = pd.merge(picking_count, placing_count, on='date', how='outer')

In [27]:
activity_counts

,date,picking_count,placing_count
0,2024-01-16,40,40
1,2024-01-17,10,9
2,2024-01-18,37,38


In [28]:
# Merge total duration and activity counts
final_output = pd.merge(total_duration, activity_counts, on='date', how='outer')

In [29]:
final_output

,date,total_inside_duration,total_outside_duration,picking_count,placing_count
0,2024-01-16,717.000000,NaN,40,40
1,2024-01-17,190.833333,NaN,10,9
2,2024-01-18,559.833333,357.833333,37,38


In [30]:
# Convert date column back to string for output consistency
final_output['date'] = final_output['date'].astype(str)

In [31]:
final_output

,date,total_inside_duration,total_outside_duration,picking_count,placing_count
0,2024-01-16,717.000000,NaN,40,40
1,2024-01-17,190.833333,NaN,10,9
2,2024-01-18,559.833333,357.833333,37,38


In [32]:
## saving the output file
final_output.to_excel('final_output.xlsx', index=False)